# Chinook SQL Data Analysis Project

## Project Overview
This project analyzes the Chinook music store database using SQL queries
executed inside a Jupyter Notebook environment.

The goal is to extract business insights related to:
- Sales performance
- Customer behavior
- Product popularity


 ## ----------------------------------

## Business Context

Chinook is a digital music store.
The management wants to understand:
- How the business is performing
- Who are the most valuable customers
- Which artists, genres, and tracks generate the most revenue

This analysis answers these questions using structured SQL queries.


In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("Data/Chinook_Sqlite.sqlite")


## Data Quality Check

### Analytical Question
Are there any missing or duplicated entries in the main tables?

Ensuring data integrity is crucial before performing any analysis.
This helps guarantee that the insights derived are accurate and reliable.


In [ ]:
# التحقق من القيم الفارغة في الجداول الرئيسية
tables = ["Customer", "Invoice", "InvoiceLine", "Track", "Album", "Artist", "Genre"]

for table in tables:
    query = f"""
    SELECT 
        COUNT(*) AS total_rows,
        SUM(CASE WHEN rowid IS NULL THEN 1 ELSE 0 END) AS missing_ids
    FROM {table};
    """
    df = pd.read_sql_query(query, conn)
    print(f"Table: {table}")
    display(df)
    print("\n")


 ## ------------------------------------------

## 1. Basic Business Metrics

Before diving into detailed analysis, we start with high-level metrics
to understand the overall scale of the business.


### Question:
How many customers, invoices, and total revenue does the business have?


In [16]:
pd.read_sql_query("SELECT COUNT(*) AS total_customers FROM Customer;", conn)

,total_customers
0,59


### Insight
The database contains **59 customers** in total.

This indicates a relatively small customer base, which suggests that
customer retention and repeat purchases are likely critical drivers
for revenue growth in this business.


 ## ----------------------------------------------

In [17]:
pd.read_sql_query("SELECT COUNT(*) AS total_invoices FROM Invoice;", conn)

,total_invoices
0,412


### Insight
A total of **412 invoices** have been recorded in the database.

When compared to the total number of customers, this suggests that
many customers have made multiple purchases, indicating repeat
buying behavior rather than one-time transactions.


 ## ----------------------------------------------

In [18]:
pd.read_sql_query("SELECT ROUND(SUM(Total), 2) AS total_revenue FROM Invoice;", conn)

,total_revenue
0,2328.6


### Insight
The total revenue generated by the store is **$2,328.60**.

Given the relatively small customer base, this revenue level highlights
the importance of understanding purchasing frequency and customer
lifetime value in subsequent analyses.


 ## ---------------------------------------------

In [19]:
pd.read_sql_query("SELECT ROUND(AVG(Total), 2) AS avg_invoice_value FROM Invoice;", conn)

,avg_invoice_value
0,5.65


### Insight
The average invoice value is **$5.65**.

This indicates that individual purchases are relatively low in value,
so total revenue depends heavily on repeat purchases and transaction
volume rather than high-priced single sales.


 ## --------------------------------------------

## 2. Sales Analysis

### Question:
Which artists generate the highest revenue for the company?


In [20]:
pd.read_sql_query("SELECT ar.Name AS artist_name, ROUND(SUM(il.UnitPrice * il.Quantity), 2) AS revenue FROM InvoiceLine il JOIN Track t ON il.TrackId = t.TrackId JOIN Album al ON t.AlbumId = al.AlbumId JOIN Artist ar ON al.ArtistId = ar.ArtistId GROUP BY ar.Name ORDER BY revenue DESC LIMIT 10;", conn)

,artist_name,revenue
0,Iron Maiden,138.60
1,U2,105.93
2,Metallica,90.09
3,Led Zeppelin,86.13
4,Lost,81.59
5,The Office,49.75
6,Os Paralamas Do Sucesso,44.55
7,Deep Purple,43.56
8,Faith No More,41.58
9,Eric Clapton,39.60


### Insight
The top 10 artists contribute disproportionately to total revenue.
For example, **Iron Maiden** alone generated **$138.60**, followed by U2 and Metallica.

This indicates that revenue is highly concentrated among a few artists,
highlighting the importance of focusing on popular artists for
promotions and inventory decisions.


 ## -----------------------------------------

In [21]:
pd.read_sql_query("""SELECT g.Name AS genre,
       ROUND(SUM(il.UnitPrice * il.Quantity), 2) AS revenue
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
JOIN Genre g ON t.GenreId = g.GenreId
GROUP BY g.Name
ORDER BY revenue DESC
LIMIT 10;""", conn)

,genre,revenue
0,Rock,826.65
1,Latin,382.14
2,Metal,261.36
3,Alternative & Punk,241.56
4,TV Shows,93.53
5,Jazz,79.20
6,Blues,60.39
7,Drama,57.71
8,R&B/Soul,40.59
9,Classical,40.59


### Insight
The top genres show that **Rock** dominates revenue (**$826.65**),
followed by Latin and Metal.

This concentration indicates that a few popular genres drive most sales,
which can inform targeted marketing strategies and playlist curation.


 ## -----------------------------------------

In [22]:
pd.read_sql_query("""SELECT BillingCountry,
       ROUND(SUM(Total), 2) AS revenue
FROM Invoice
GROUP BY BillingCountry
ORDER BY revenue DESC
LIMIT 10;""", conn)

,BillingCountry,revenue
0,USA,523.06
1,Canada,303.96
2,France,195.10
3,Brazil,190.10
4,Germany,156.48
5,United Kingdom,112.86
6,Czech Republic,90.24
7,Portugal,77.24
8,India,75.26
9,Chile,46.62


### Insight
The **USA** leads with **$523.06**, followed by Canada and France.

This indicates that North America is the primary revenue source,
highlighting the importance of focusing business efforts
on high-performing international markets.


 ## --------------------------------------

## 3. Customer Analysis

Understanding customer behavior helps identify:
- High-value customers
- Purchase frequency patterns


In [23]:
pd.read_sql_query("""SELECT c.FirstName || ' ' || c.LastName AS customer_name,
       ROUND(SUM(i.Total), 2) AS total_spent
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY total_spent DESC
LIMIT 10;""", conn)

,customer_name,total_spent
0,Helena Holý,49.62
1,Richard Cunningham,47.62
2,Luis Rojas,46.62
3,Ladislav Kovács,45.62
4,Hugh O'Reilly,45.62
5,Frank Ralston,43.62
6,Julia Barnett,43.62
7,Fynn Zimmermann,43.62
8,Astrid Gruber,42.62
9,Victor Stevens,42.62


### Insight
The top 10 customers together have spent between **$42.62 and $49.62**.

This shows that a small group of customers contributes significantly
to total revenue, emphasizing the importance of customer
segmentation and retention strategies.


 ## -----------------------------------------

In [24]:
pd.read_sql_query("""SELECT c.FirstName || ' ' || c.LastName AS customer_name,
       COUNT(i.InvoiceId) AS num_invoices,
       ROUND(AVG(i.Total), 2) AS avg_invoice
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY num_invoices DESC
LIMIT 10;""", conn)

,customer_name,num_invoices,avg_invoice
0,Luís Gonçalves,7,5.66
1,Leonie Köhler,7,5.37
2,François Tremblay,7,5.66
3,Bjørn Hansen,7,5.66
4,František Wichterlová,7,5.80
5,Helena Holý,7,7.09
6,Astrid Gruber,7,6.09
7,Daan Peeters,7,5.37
8,Kara Nielsen,7,5.37
9,Eduardo Martins,7,5.37


### Insight
Several customers have made **7 purchases each**, with average invoice
values ranging from **$5.37 to $7.09**.

This indicates a small number of highly engaged customers
who repeatedly contribute to revenue, emphasizing
the value of customer loyalty programs.


 ## -------------------------------------

 ## Analytical Question
Which customers have made only a single purchase?

In [25]:
pd.read_sql_query("""SELECT c.FirstName || ' ' || c.LastName AS customer_name,
       COUNT(i.InvoiceId) AS num_invoices
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
HAVING num_invoices = 1;""", conn)

,customer_name,num_invoices


### Insight
No customers in the dataset have only a single purchase.

This suggests that all customers have engaged in repeat transactions,
which is positive for customer retention and indicates consistent
purchasing behavior.


 ## --------------------------------------

## 11. Top 10 Tracks by Sales Volume

### Analytical Question
Which tracks have been sold the most in terms of quantity?

Analyzing track sales helps identify the most popular products
and can guide inventory, promotions, and marketing strategies.


In [26]:
pd.read_sql_query("""SELECT t.Name AS track_name,
       COUNT(il.InvoiceLineId) AS times_sold,
       ROUND(SUM(il.UnitPrice * il.Quantity), 2) AS revenue
FROM Track t
JOIN InvoiceLine il ON t.TrackId = il.TrackId
GROUP BY t.TrackId
ORDER BY times_sold DESC
LIMIT 10;""", conn)

,track_name,times_sold,revenue
0,Balls to the Wall,2,1.98
1,Inject The Venom,2,1.98
2,Snowballed,2,1.98
3,Overdose,2,1.98
4,Deuces Are Wild,2,1.98
5,Not The Doctor,2,1.98
6,Por Causa De Você,2,1.98
7,Welcome Home (Sanitarium),2,1.98
8,Snowblind,2,1.98
9,Cornucopia,2,1.98


### Insight
The top 10 tracks have each been sold **2 times**, generating
**$1.98** in revenue per track.

This indicates that sales are distributed across many tracks
without a single dominant best-seller, suggesting a diverse
catalog with relatively low individual track sales.


 ## -----------------------------------------

In [27]:
pd.read_sql_query("""WITH customer_totals AS (
    SELECT c.CustomerId,
           SUM(i.Total) AS total_spent
    FROM Customer c
    JOIN Invoice i ON c.CustomerId = i.CustomerId
    GROUP BY c.CustomerId
)
SELECT SUM(total_spent) AS top_20_percent_revenue
FROM customer_totals
WHERE CustomerId IN (
    SELECT CustomerId
    FROM customer_totals
    ORDER BY total_spent DESC
    LIMIT (SELECT ROUND(COUNT(*) * 0.2) FROM customer_totals)
);""", conn)

,top_20_percent_revenue
0,533.44


### Insight
The top 20% of customers contribute **$533.44** in revenue,
which is a substantial portion of the total sales.

This confirms that revenue is concentrated among a small group of
customers, highlighting the importance of identifying and retaining
high-value clients.


 ## ------------------------------------------

In [28]:
pd.read_sql_query("""SELECT BillingCountry,
       COUNT(InvoiceId) AS num_invoices,
       ROUND(SUM(Total)/COUNT(InvoiceId), 2) AS avg_invoice
FROM Invoice
GROUP BY BillingCountry
ORDER BY avg_invoice DESC;""", conn)

,BillingCountry,num_invoices,avg_invoice
0,Chile,7,6.66
1,Ireland,7,6.52
2,Hungary,7,6.52
3,Czech Republic,14,6.45
4,Austria,7,6.09
5,Finland,7,5.95
6,Netherlands,7,5.80
7,India,13,5.79
8,USA,91,5.75
9,Norway,7,5.66


### Insight
Countries like **Chile**, **Ireland**, and **Hungary** have the highest
average invoice values (**$6.52–$6.66**), while larger markets like the
USA and Canada have slightly lower averages.

This suggests that smaller markets may have fewer transactions but
higher spending per invoice, indicating different purchasing behaviors
across countries.


 ## -----------------------------------------

## Revenue Trend Over Time

### Analytical Question
How has the revenue of the Chinook music store changed over time?

Analyzing revenue trends helps identify:
- Seasonal patterns
- Growth or decline in sales
- Opportunities for forecasting and planning


In [ ]:
# تحويل تاريخ الفاتورة إلى نوع datetime
invoice_df = pd.read_sql_query("SELECT InvoiceDate, Total FROM Invoice;", conn)
invoice_df['InvoiceDate'] = pd.to_datetime(invoice_df['InvoiceDate'])

# حساب الإيرادات الشهرية
monthly_revenue = invoice_df.groupby(invoice_df['InvoiceDate'].dt.to_period('M'))['Total'].sum().reset_index()
monthly_revenue['InvoiceDate'] = monthly_revenue['InvoiceDate'].dt.to_timestamp()

# عرض الإيرادات الشهرية
monthly_revenue.head(12)



## Correlation Analysis and Patterns

### Analytical Question
Are there patterns or correlations between variables such as:
- Genre and Revenue
- Country and Average Invoice

Identifying relationships helps make strategic business decisions,
optimize marketing, and improve product offerings.


In [ ]:
# Genre vs Total Revenue per Customer
genre_revenue_query = """
SELECT g.Name AS Genre,
       ROUND(SUM(il.UnitPrice * il.Quantity), 2) AS Total_Revenue
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
JOIN Genre g ON t.GenreId = g.GenreId
GROUP BY g.Name
ORDER BY Total_Revenue DESC;
"""
genre_revenue_df = pd.read_sql_query(genre_revenue_query, conn)

genre_revenue_df.head(10)




In [ ]:
# Country vs Average Invoice
country_avg_invoice_query = """
SELECT BillingCountry,
       ROUND(SUM(Total)/COUNT(InvoiceId), 2) AS Avg_Invoice
FROM Invoice
GROUP BY BillingCountry
ORDER BY Avg_Invoice DESC;
"""
country_avg_invoice_df = pd.read_sql_query(country_avg_invoice_query, conn)
country_avg_invoice_df.head(10)

In [29]:
conn.close()

## Final Conclusions

- The business revenue is driven by a limited set of artists and customers
- Customer behavior follows a Pareto distribution
- SQL is effective for extracting business insights directly from relational databases

This project demonstrates the ability to:
- Explore database schemas
- Write analytical SQL queries
- Translate raw data into actionable insights
